In [3]:
import pandas as pd

# Cargar los datos
data = pd.read_csv('angelsdata.csv')

# Verificar los nombres de las columnas
print(data.columns)


/tmp/ipykernel_29215/2191803649.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('angelsdata.csv')


Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT',
       'LON'],
      dtype='object')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Cargar los datos
data = pd.read_csv('angelsdata.csv')

# Preprocesamiento de datos
data['DATE OCC'] = pd.to_datetime(data['DATE OCC'])  # Asegúrate de usar la columna correcta
data['HOUR'] = data['DATE OCC'].dt.hour
data['DAY_OF_WEEK'] = data['DATE OCC'].dt.dayofweek
data['MONTH'] = data['DATE OCC'].dt.month

# Seleccionar características y etiquetas, usando 'Crm Cd Desc' en lugar de 'PRIMARY DESCRIPTION'
X = data[['HOUR', 'DAY_OF_WEEK', 'MONTH']].values
y = data['Crm Cd Desc']

# Codificar las etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar características
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Ajustar las dimensiones para LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Construir la red neuronal recurrente 
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], 1), activation='relu', return_sequences=True),
    LSTM(32, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Ajustar el modelo
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


/tmp/ipykernel_29215/3870284202.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('angelsdata.csv')
/tmp/ipykernel_29215/3870284202.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['DATE OCC'] = pd.to_datetime(data['DATE OCC'])  # Asegúrate de usar la columna correcta
/home/user/jupyter_mineria/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
16980/16980 ━━━━━━━━━━━━━━━━━━━━ 73s 4ms/step - accuracy: 0.0170 - loss: nan - val_accuracy: 0.0026 - val_loss: nan
Epoch 2/20
16980/16980 ━━━━━━━━━━━━━━━━━━━━ 76s 4ms/step - accuracy: 0.0030 - loss: nan - val_accuracy: 0.0026 - val_loss: nan
Epoch 3/20
16980/16980 ━━━━━━━━━━━━━━━━━━━━ 94s 5ms/step - accuracy: 0.0027 - loss: nan - val_accuracy: 0.0026 - val_loss: nan
Epoch 4/20
16980/16980 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step - accuracy: 0.0028 - loss: nan - val_accuracy: 0.0026 - val_loss: nan
Epoch 5/20
16980/16980 ━━━━━━━━━━━━━━━━━━━━ 89s 5ms/step - accuracy: 0.0028 - loss: nan - val_accuracy: 0.0026 - val_loss: nan
Epoch 6/20
16980/16980 ━━━━━━━━━━━━━━━━━━━━ 103s 6ms/step - accuracy: 0.0029 - loss: nan - val_accuracy: 0.0026 - val_loss: nan
Epoch 7/20
16980/16980 ━━━━━━━━━━━━━━━━━━━━ 87s 5ms/step - accuracy: 0.0028 - loss: nan - val_accuracy: 0.0026 - val_loss: nan
Epoch 8/20
16980/16980 ━━━━━━━━━━━━━━━━━━━━ 160s 6ms/step - accuracy: 0.0029 - loss: nan - val_accuracy: 0.002

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.preprocessing import StandardScaler

# Cargar los datos
data = pd.read_csv('angelsdata.csv')

# Preprocesamiento de datos
data['DATE OCC'] = pd.to_datetime(data['DATE OCC'])
data = data.sort_values(by='DATE OCC')  # Ordenar cronológicamente

# Seleccionar características y etiquetas usando 'Crm Cd Desc' en lugar de 'PRIMARY DESCRIPTION'
X = data[['DATE OCC']].values
y = data['Crm Cd Desc']  # Usar la descripción del crimen como la variable objetivo

# Codificar las etiquetas
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Escalar características
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Ajustar las dimensiones para la entrada de CNN (reshape)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Construir la red neuronal convolucional (CNN)
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Ajustar el modelo
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))
